In [1]:
'''
Resnet for face recognition

MLAI Project

yidongh@andrew.cmu.edu

2019-11-28
'''
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets
import torch.optim as optim

from torch.utils.data import DataLoader, Dataset

cuda = torch.cuda.is_available()
cuda

True

## MobileNet


In [2]:
%run MobileNetV2_2.py

## Load Data

In [4]:
def parse_data(datadir):
    img_list = []
    ID_list = []
    uniqueID_list = []
    for root, directories, filenames in os.walk(datadir):
        b = -1
        for filename in filenames:
            if filename.endswith('.jpg'):
                if(filename[0]+filename[1]=='._'):
                    filename = filename[2:]
                filei = os.path.join(root, filename)
                img_list.append(filei)
                ID_list.append(root.split('/')[-1])
                a = root.split('/')[-1]
                if a!=b:
                    uniqueID_list.append(a)
                b=a
                
    class_n = len(uniqueID_list)
    target_dict = dict(zip(uniqueID_list, range(class_n)))
    reverse_dict = dict(zip(range(class_n),uniqueID_list))
    label_list = [target_dict[ID_key] for ID_key in ID_list]
    
    print('{}\t\t{}\n{}\t\t{}'.format('#Images', '#Labels', len(img_list), len(set(label_list))))
    return img_list, label_list, class_n, reverse_dict

## Load Network

In [5]:
network = MobileNetV2(num_classes = 2300)
network.load_state_dict(torch.load('model_epoch22.pth'))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
network.to(device)
network.eval()

MobileNetV2(
  (classifier): Sequential(
    (0): Dropout(p=0.3, inplace=False)
    (1): Linear(in_features=512, out_features=2300, bias=True)
  )
  (feature): Sequential(
    (0): Conv2d(3, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU6(inplace=True)
    (3): InvertedResidual(
      (conv): Sequential(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
        (3): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (4): InvertedResidual(
      (conv): Sequential(
        (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_runn

In [6]:
test_img_list  = []
for root, dictionaries, filenames in os.walk('test_classification/medium/'):
    for filename in filenames:
        img_path = os.path.join(root, filename)
        test_img_list.append(img_path)
        
test_img_list.sort()


In [7]:
print(len(test_img_list))

4600


In [8]:
class testDataset(Dataset):
    def __init__(self, file_list):
        self.file_list = file_list
    
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, index):
        img = Image.open(self.file_list[index])
        img = torchvision.transforms.ToTensor()(img)
        
        return img

In [9]:
testset = testDataset(test_img_list)
test_dataLoader = DataLoader(testset, batch_size=1, shuffle=False, num_workers=3)


In [10]:
print(len(testset))

4600


In [11]:
test_embedding_resnet = []
for batch_num, (feats) in enumerate(test_dataLoader):
    feats = feats.to(device)
    output = network(feats)
    embedding = np.copy(output.cpu().detach().numpy())
    test_embedding_resnet.append(embedding)
    
    del feats
    
    

In [12]:
test_embedding_save = np.zeros((1,512))
for i in range(len(test_embedding_resnet)):
    test_embedding_save = np.concatenate((test_embedding_save,test_embedding_resnet[i]),axis = 0)
    
test_embedding_save = test_embedding_save[1:,:]
print(test_embedding_save.shape)

(4600, 512)


In [13]:
print(len(test_embedding_resnet))

4600


In [14]:
np.save('MobileNet_test_embedding.npy',test_embedding_save)